In [1]:
import trafilatura
import requests
import lxml
from elasticsearch import Elasticsearch, helpers


In [2]:
cafef = {
    "life": ["https://cafef.vn/song.chn",
             "https://cafef.vn/lifestyle.chn"],
    "news": ["https://cafef.vn/thoi-su.chn"],
    "world": ["https://cafef.vn/tai-chinh-quoc-te.chn"]
}


In [3]:
cafefLinks = {
    "life": [],
    "news": [],
    "world": []
}
for key, value in cafef.items():
    for url in value:
        response = requests.get(url)
        tree = lxml.html.fromstring(response.content)
        title_elem = tree.cssselect('h3 a')
        for elem in title_elem:
            cafefLinks[key].append("https://cafef.vn"+elem.get('href'))

print(cafefLinks)

{'life': ['https://cafef.vn/so-phan-chim-noi-cua-vien-da-minh-chau-gia-3000-ty-dat-trong-mieng-tu-hi-thai-hau-luc-qua-doi-noi-cat-giu-hien-tai-vo-cung-ly-ky-2022040508350304.chn', 'https://cafef.vn/toi-do-gay-chan-dong-pho-wall-bill-hwang-kiem-15-ty-usd-trong-9-nam-sau-cung-toan-bo-tai-san-boc-hoi-trong-2-ngay-tro-thanh-ke-no-nan-chong-chat-20220405103701801.chn', 'https://cafef.vn/4-giac-mo-ky-la-duoc-tin-la-diem-bao-cho-viec-than-tai-chieu-co-tai-loc-nhay-vot-20220302115701746.chn', 'https://cafef.vn/dinh-cao-tri-tue-gia-cat-luong-ngam-7-hat-gao-sau-khi-qua-doi-bai-binh-bo-tran-doa-tu-ma-y-mot-phen-hon-sieu-phach-lac-20220405203157503.chn', 'https://cafef.vn/loai-rau-gia-vai-nghin-mot-bo-dang-ban-day-cho-vua-bo-mau-vua-chua-2-loai-vitamin-san-xuat-collagen-chi-em-an-deu-se-giam-can-chong-lao-hoa-20220405202131672.chn', 'https://cafef.vn/nu-nhan-vien-van-phong-23-tuoi-dot-nhien-dau-tai-liet-mat-sau-khi-tang-ca-ve-khuya-bac-si-canh-bao-nguoi-tre-nen-can-trong-trong-thoi-tiet-giao-mua-2

In [4]:
cafefContent = {
    "life": [],
    "news": [],
    "world": []
}
for key, value in cafefLinks.items():
    for url in value:
        if(url.startswith('https://cafef.vn/')):
            id = url.split('-')[-1].replace('.chn', '')
            downloaded = trafilatura.fetch_url(url)
            tree = lxml.html.fromstring(str(downloaded))
            date_elem = tree.cssselect(".pdate")[0]
            content = trafilatura.extract(downloaded)
            content = content.replace('\n', ' ')
            item = {"domain": "https://cafef.vn", "URL": url,"content": content, "date": date_elem.text, "id": id}
            cafefContent[key].append(item)

print(len(cafefContent["life"]) + len(cafefContent["news"]) + len(cafefContent["world"]))

2022040508350304
20220405103701801
20220302115701746
20220405203157503
20220405202131672
20220405140929974
20220405200825003
20220405145257116
20220405150313144
20220405151217812
20220405155232314
20220405145555582
20220405135636484
20220405074021245
20220405011712148
20220404153955688
2022040520414739
20220405201814585
2022040520252149


KeyboardInterrupt: 

In [ ]:
client = Elasticsearch("http://localhost:9200")

client.info()


In [ ]:
def gendata():
    for key, value in cafefContent.items():
        for item in value:
            yield {
                "_index": "journal",
                "_type": key,
                "_id": item['id'],
                "doc": item
            }


helpers.bulk(es, gendata())
